In [15]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

In [16]:
import itertools

In [17]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

In [11]:
df = pd.read_csv('../Downloads/demo_LTC-USD_20190926.csv')
df['system_time'] = pd.to_datetime(df['system_time'])
df['date'] = df['system_time'].dt.date

In [12]:
df

,system_time,midpoint,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4,...,asks_market_notional_6,asks_market_notional_7,asks_market_notional_8,asks_market_notional_9,asks_market_notional_10,asks_market_notional_11,asks_market_notional_12,asks_market_notional_13,asks_market_notional_14,date
0,2019-09-26 20:36:38.067000+00:00,53.825,0.01,0.0,0.000000,-0.000093,-0.000650,-0.001022,-0.001393,-0.001579,...,0,0.0,0,0,0.0,0,0,0,0,2019-09-26
1,2019-09-26 20:36:39.067000+00:00,53.825,0.01,0.0,5.500119,-0.000093,-0.000650,-0.001022,-0.001393,-0.001579,...,0,0.0,0,0,0.0,0,0,0,0,2019-09-26
2,2019-09-26 20:36:40.067000+00:00,53.825,0.01,0.0,0.000000,-0.000093,-0.000650,-0.001022,-0.001393,-0.001579,...,0,0.0,0,0,0.0,0,0,0,0,2019-09-26
3,2019-09-26 20:36:41.067000+00:00,53.825,0.01,0.0,0.000000,-0.000093,-0.000650,-0.001022,-0.001393,-0.001579,...,0,0.0,0,0,0.0,0,0,0,0,2019-09-26
4,2019-09-26 20:36:42.067000+00:00,53.825,0.01,0.0,0.000000,-0.000093,-0.000650,-0.001022,-0.001393,-0.001579,...,0,0.0,0,0,0.0,0,0,0,0,2019-09-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91397,2019-09-27 21:59:55.067000+00:00,56.475,0.03,0.0,0.000000,-0.000266,-0.000620,-0.000797,-0.000974,-0.001151,...,0,0.0,0,0,0.0,0,0,0,0,2019-09-27
91398,2019-09-27 21:59:56.067000+00:00,56.475,0.03,0.0,0.000000,-0.000266,-0.000620,-0.000797,-0.000974,-0.001328,...,0,0.0,0,0,0.0,0,0,0,0,2019-09-27
91399,2019-09-27 21:59:57.067000+00:00,56.475,0.03,0.0,0.000000,-0.000266,-0.000620,-0.000797,-0.000974,-0.001328,...,0,0.0,0,0,0.0,0,0,0,0,2019-09-27
91400,2019-09-27 21:59:58.067000+00:00,56.475,0.03,0.0,0.000000,-0.000266,-0.000620,-0.000797,-0.000974,-0.001505,...,0,0.0,0,0,0.0,0,0,0,0,2019-09-27


In [18]:
TRAIN_START_DATE = '2010-01-01'
TRAIN_END_DATE = '2021-10-01'
TRADE_START_DATE = '2021-10-01'
TRADE_END_DATE = '2023-03-01'
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

YF deprecation warning: set proxy via new config function: yf.set_config(proxy=proxy)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (97013, 8)


In [19]:
print(config_tickers.DOW_30_TICKER)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [20]:
df.shape

(97013, 8)

In [21]:
df.sort_values(['date','tic'],ignore_index=True).head()

Price,date,close,high,low,open,volume,tic,day
0,2010-01-04,6.431896,6.446622,6.382907,6.414464,493729600,AAPL,0
1,2010-01-04,39.913250,40.016973,39.111113,39.159517,5277400,AMGN,0
2,2010-01-04,32.720371,32.864303,32.296575,32.632416,6894300,AXP,0
3,2010-01-04,43.777546,43.941185,42.702198,43.419098,6186700,BA,0
4,2010-01-04,39.547806,39.980096,38.845333,38.939899,7325600,CAT,0


In [22]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators


[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (3310, 8)
Successfully added vix
Successfully added turbulence index


In [9]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [10]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2010-01-04,AAPL,6.431897,6.446623,6.382908,6.414465,493729600.0,0.0,0.0,6.453181,6.421732,100.0,66.666667,100.0,6.431897,6.431897,20.040001,0.0
1,2010-01-04,AMGN,39.913254,40.016977,39.111116,39.159521,5277400.0,0.0,0.0,6.453181,6.421732,100.0,66.666667,100.0,39.913254,39.913254,20.040001,0.0
2,2010-01-04,AXP,32.720364,32.864295,32.296567,32.632408,6894300.0,0.0,0.0,6.453181,6.421732,100.0,66.666667,100.0,32.720364,32.720364,20.040001,0.0
3,2010-01-04,BA,43.777554,43.941193,42.702205,43.419105,6186700.0,0.0,0.0,6.453181,6.421732,100.0,66.666667,100.0,43.777554,43.777554,20.040001,0.0
4,2010-01-04,CAT,39.547806,39.980096,38.845333,38.939899,7325600.0,0.0,0.0,6.453181,6.421732,100.0,66.666667,100.0,39.547806,39.547806,20.040001,0.0
5,2010-01-04,CRM,18.571924,18.748161,18.415545,18.519798,7906000.0,0.0,0.0,6.453181,6.421732,100.0,66.666667,100.0,18.571924,18.571924,20.040001,0.0
6,2010-01-04,CSCO,16.255333,16.354089,15.807636,15.873474,59853700.0,0.0,0.0,6.453181,6.421732,100.0,66.666667,100.0,16.255333,16.255333,20.040001,0.0
7,2010-01-04,CVX,42.603535,42.678977,42.118550,42.140101,10173800.0,0.0,0.0,6.453181,6.421732,100.0,66.666667,100.0,42.603535,42.603535,20.040001,0.0
8,2010-01-04,DIS,27.593042,28.178115,27.420963,27.963015,13700400.0,0.0,0.0,6.453181,6.421732,100.0,66.666667,100.0,27.593042,27.593042,20.040001,0.0
9,2010-01-04,GS,132.649887,133.546583,129.913805,130.327672,9135000.0,0.0,0.0,6.453181,6.421732,100.0,66.666667,100.0,132.649887,132.649887,20.040001,0.0


In [11]:
mvo_df = processed_full.sort_values(['date','tic'],ignore_index=True)[['date','tic','close']]

In [12]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

85753
10237


In [13]:
train.tail()

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
2956,2021-09-30,UNH,371.414154,383.533544,371.148004,381.632464,3779900.0,3.0,-4.215702,406.339796,374.984609,40.895414,-232.790874,41.349557,393.482302,392.988889,23.139999,24.871882
2956,2021-09-30,V,216.564880,222.437161,216.448217,221.260767,7128500.0,3.0,-1.512930,224.805867,212.899234,44.078994,-54.737424,19.487730,221.008639,227.598414,23.139999,24.871882
2956,2021-09-30,VZ,42.742081,43.137767,42.734168,43.129855,18736600.0,3.0,-0.203777,43.718475,42.508790,41.824895,-103.383611,21.770769,43.259641,43.717979,23.139999,24.871882
2956,2021-09-30,WBA,38.408340,39.943042,38.302217,39.828757,6449400.0,3.0,-0.221534,42.405393,38.093748,44.613721,-107.978201,0.109723,40.094060,39.195162,23.139999,24.871882
2956,2021-09-30,WMT,44.113533,44.857304,44.072390,44.512323,22457700.0,3.0,-0.503907,47.367513,43.957437,40.165852,-151.607674,45.326129,46.132792,45.776751,23.139999,24.871882


In [14]:
trade.head()

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2021-10-01,AAPL,139.809128,140.073755,136.339633,139.074064,94639600.0,4.0,-1.684269,153.630011,135.585275,46.927701,-142.139008,41.705495,145.511608,144.619407,21.1,120.123159
0,2021-10-01,AMGN,189.702744,190.314633,186.935954,189.410100,2629400.0,4.0,-2.882433,198.005813,185.546302,40.408590,-96.444300,35.031857,194.016102,202.040364,21.1,120.123159
0,2021-10-01,AXP,165.851669,166.976791,160.645557,160.664631,3956000.0,4.0,2.217001,169.902242,145.535369,56.265109,117.704000,15.682256,157.221237,159.408883,21.1,120.123159
0,2021-10-01,BA,226.000000,226.720001,220.600006,222.850006,9113600.0,4.0,0.730320,226.909442,205.727561,51.614047,116.649440,2.027170,217.175334,221.968500,21.1,120.123159
0,2021-10-01,CAT,180.580658,182.011693,177.709287,179.251827,3695500.0,4.0,-3.497998,197.692313,174.339364,41.999424,-111.971181,36.144648,189.292056,192.682353,21.1,120.123159


In [15]:
INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [16]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 29, State Space: 291


In [17]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [18]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [20]:
agent = DRLAgent(env = env_train)

if_using_a2c = True
if_using_ddpg = True
if_using_ppo = True
if_using_td3 = True
if_using_sac = True

In [21]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

if if_using_a2c:
  # set up logger
  tmp_path = RESULTS_DIR + '/a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cuda device
Logging to results/a2c


/home/supylity/miniconda3/envs/FinRl_new/lib/python3.11/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run A2C on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [ ]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000) if if_using_a2c else None

---------------------------------------
| time/                 |             |
|    fps                | 239         |
|    iterations         | 100         |
|    time_elapsed       | 2           |
|    total_timesteps    | 500         |
| train/                |             |
|    entropy_loss       | -41.2       |
|    explained_variance | 0           |
|    learning_rate      | 0.0007      |
|    n_updates          | 99          |
|    policy_loss        | -61.1       |
|    reward             | -0.35417625 |
|    std                | 1           |
|    value_loss         | 2.97        |
---------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 248        |
|    iterations         | 200        |
|    time_elapsed       | 4          |
|    total_timesteps    | 1000       |
| train/                |            |
|    entropy_loss       | -41.2      |
|    explained_variance | 0.00175    |
|    lear